In [3]:
import sys
import os
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')

import yaml
import numpy as np
import random
import json
import argparse

import hydra
import multiprocessing as mp
from omegaconf import DictConfig, OmegaConf
import orbax

import jax.numpy as jnp
from jax import grad, jit, vmap
import jax
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)


from models.Recurrent_Linear_Transformers.src.trainers.trainers_control import ControlTrainer

cpu


In [4]:
task_to_trainer={
    'minigrid_pixel':ControlTrainer,
    'minigrid_onehot':ControlTrainer,
    'tmazev1':ControlTrainer,
    'tmazev2':ControlTrainer,
    'tmaze_ours':ControlTrainer,
    'memory_gym':ControlTrainer,
    'popgym':ControlTrainer,
    'procgen':ControlTrainer,
    'mujoco':ControlTrainer,
    'memorymaze':ControlTrainer,
    'vizdoom_two_colors':ControlTrainer,
}

In [5]:
# with open("../../models/Recurrent_Linear_Transformers/config/vizdoom_two_colors/arelit.yaml") as f:
#     config = yaml.load(f, Loader=yaml.FullLoader)

config = OmegaConf.load("../../models/Recurrent_Linear_Transformers/config/vizdoom_two_colors/arelit.yaml")

In [6]:
print(OmegaConf.to_yaml(config))

tags: null
project_name: AReLiT-ViZDoom-Two-Colors-Pillar
seed: 1
steps: 200000000
log_interval: 10000
eval_interval: 1000000
eval_episodes: 10
videos: true
record_fps: 30
save_interval: 1000000
save_dir: ./checkpoints/arelit_doom_with_pillar
save_tag: arelit_doom_with_pillar
task:
  task: vizdoom_two_colors
  env_name: doom_with_pillar
  max_episode_steps: 4200
  seed: null
trainer:
  agent: ppo
  seq_model:
    name: arelit
    n_layers: 4
    d_model: 512
    d_head: 64
    d_ffc: 512
    n_heads: 8
    eta: 4
    r: 7
    reset_hidden_on_terminate: true
  env_pool: async
  d_actor: 256
  d_critic: 256
  num_envs: 32
  rollout_len: 200
  sequence_length: 100
  anneal_lr: false
  gamma: 0.99
  gae_lambda: 0.95
  num_minibatches: 8
  update_epochs: 1
  norm_adv: true
  clip_coef: 0.1
  ent_coef:
    initial: 0.003
    final: null
    max_decay_steps: 10000
    power: 1
  vf_coef: 0.5
  max_grad_norm: 4.0
  optimizer:
    learning_rate:
      initial: 0.00025
      final: null
      ma

In [7]:
key=jax.random.PRNGKey(config.seed)
np.random.seed(config.seed)
random.seed(config.seed)
trainer_config=config.trainer
env_config=config.task

#Train the model
kwargs={'global_args': config,
        'trainer_config': trainer_config,
        'env_config' :env_config,
        'seed':config.seed,
        'key':key,
        'wandb_run': 1}

trainer=task_to_trainer[env_config['task']](**kwargs)

/home/jovyan/Egor_C/REPOSITORIES/Memory-RL-Codebase/tests/ViZDoom-Two-Colors/../../models/Recurrent_Linear_Transformers/src/trainers/trainers_control.py:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  eval_seeds=int(np.random.randint(0,9999,size=1,dtype=int))
/home/user/conda/envs/memory_rl_relit/lib/python3.9/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/jovyan/Egor_C/REPOSITORIES/Memory-RL-Codebase/tests/ViZDoom-Two-Colors/../../models/Recurrent_Linear_Transformers/src/agents/base_agent.py:69: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  self.h_tickmi

In [8]:
checkpoint_dir = '/home/jovyan/Egor_C/REPOSITORIES/Memory-RL-Codebase/checkpoints/arelit_doom_with_pillar/arelit_doom_with_pillar'
trainer.checkpoint_dir = checkpoint_dir

In [9]:
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
options = orbax.checkpoint.CheckpointManagerOptions(max_to_keep=2, create=False)
trainer.manager = orbax.checkpoint.CheckpointManager(trainer.checkpoint_dir, orbax_checkpointer, options)
trainer.restore_checkpoint()

/home/user/conda/envs/memory_rl_relit/lib/python3.9/site-packages/orbax/checkpoint/type_handlers.py:1401: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(
ERROR:root:Device cuda:0 was not found in jax.local_devices().
ERROR:root:Device cuda:0 was not found in jax.local_devices().
ERROR:root:Device cuda:0 was not found in jax.local_devices().
ERROR:root:Device cuda:0 was not found in jax.local_devices().
ERROR:root:Device cuda:0 was not found in jax.local_devices().
ERROR:root:Device cuda:0 was not found in jax.local_devices().
ERROR:root:Device cuda:0 was not found in jax.local_devices().
ERROR:root:Device cuda:0 was not found in jax.local_devices().
ERROR:root:Device cuda:0 was not foun

ValueError: sharding passed to deserialization should be specified, concrete and an instance of `jax.XLACompatibleSharding`. Got None

In [ ]:
trainer.agent

: 